**Objetivo de esta etapa:** Entrenar múltiples versiones del modelo MF con diferentes combinaciones de hiperparámetros y comparar su rendimiento

**Grid Search:** Prueba todas las combinaciones posibles de un conjunto de valores. Es exhaustivo pero lento.

**Random Search**: Elige combinaciones al azar durante X intentos. Mucho más rápido, y en la práctica muy efectivo.

**Algoritmos genéticos/Optuna:** Estrategias inteligentes que aprenden qué combinaciones probar. Pueden encontrar buenas configuraciones en menos tiempo.


## Grid Search

In [ ]:
from cornac.data import Reader
from cornac.eval_methods import RatioSplit


# Load MovieLens 100K dataset
reader = Reader()
ml_data = reader.read(fpath='./datasets/ml-100k/u.data', fmt='UIRT', sep='\t')

# Define the evaluation metrics
eval_method = RatioSplit(
    data=ml_data,
    test_size=0.2,
    rating_threshold=0.0,
    exclude_unknowns=True,
    verbose=True,
    seed=42
)

In [ ]:
from itertools import product   # generar todas las combinaciones posibles de varios conjuntos de valores
from cornac.models import MF
from cornac.metrics import RMSE, MAE, Precision, NDCG

import pandas as pd

# Hiperparámetros que vamos a probar
k_list = [10, 20]                 # Dimensión de los vectores latentes
lr_list = [0.01, 0.005]           # Velocidad de aprendizaje
reg_list = [0.01, 0.1]            # Regularización L2 (evita sobreajuste)

resultados = []

for k, lr, reg in product(k_list, lr_list, reg_list):
    print(f"\nEntrenando MF con k={k}, lr={lr}, reg={reg}")
    
    model = MF(
        k=k,
        learning_rate=lr,
        lambda_reg=reg,
        max_iter=50,
        verbose=False
    )
    
    model.fit(eval_method.train_set)

    metrics = eval_method.evaluate(
        model,
        metrics=[RMSE(), MAE(), Precision(k=5), NDCG(k=5)],
        user_based=True
    )
    
    res = dict(metrics[0].metric_avg_results)
    res.update({"k": k, "lr": lr, "reg": reg})
    resultados.append(res)

# Convertir resultados a DataFrame y ordenar por RMSE
df_resultados = pd.DataFrame(resultados)
df_resultados = df_resultados.sort_values(by="RMSE")

# Mostrar tabla ordenada por RMSE
print(df_resultados)

**Tabla de resultados de Grid Search de hiperparámetros sobre MF**

| k  | lr    | reg  | RMSE       | MAE        | Precision\@5 | NDCG\@5    | Train (s) | Test (s) |
| -- | ----- | ---- | ---------- | ---------- | ------------ | ---------- | --------- | -------- |
| 10 | 0.005 | 0.01 | 0.9376     | 0.7621     | **0.0881**   | **0.0902** | 0.021     | 0.711    |
| 20 | 0.005 | 0.01 | 0.9610     | 0.7800     | 0.0809       | 0.0839     | 0.083     | 0.997    |
| 10 | 0.010 | 0.01 | 0.9831     | 0.7895     | 0.0415       | 0.0407     | 0.055     | 1.241    |
| 20 | 0.010 | 0.01 | 1.0411     | 0.8415     | 0.0500       | 0.0502     | 0.021     | 0.681    |
| 20 | 0.005 | 0.10 | **0.8894** | **0.7317** | 0.0347       | 0.0297     | 0.047     | 0.675    |
| 10 | 0.005 | 0.10 | 0.8912     | 0.7334     | 0.0302       | 0.0251     | 0.065     | 0.652    |
| 10 | 0.010 | 0.10 | 0.8913     | 0.7331     | 0.0187       | 0.0166     | 0.044     | 0.644    |
| 20 | 0.010 | 0.10 | 0.8924     | 0.7332     | 0.0228       | 0.0203     | 0.035     | 0.642    |

**Mejor precisión numérica (RMSE más bajo)**

* `k=20`, `lr=0.005`, `reg=0.1`
* Mejor RMSE
* Pero baja Precision\@5 y NDCG\@5 (solo 3.4%)

Este modelo predice bien los ratings, pero no rankea bien los ítems (el orden no coincide con lo que el usuario desea).


**Mejor ranking (Precision\@5 y NDCG\@5)**

  * `k=10`, `lr=0.005`, `reg=0.01`
  * Precision\@5 = 0.0881
  * NDCG\@5 = 0.0902
  * RMSE = 0.9376 (más alto que el mínimo, pero aceptable)

Este modelo recomienda mejor el Top-N, aunque no predice los ratings con tanta precisión.

**Conclusiones**

1. No hay una sola combinación “mejor”. Depende del objetivo:

   * ¿Predecir bien el rating? → RMSE bajo.
   * ¿Que el Top-5 recomendado sea útil? → Precision\@5 alto.
2. Regularización alta (reg=0.1) mejora RMSE pero perjudica Precision\@5.
3. Más dimensiones (k=20) mejora RMSE pero no siempre mejora ranking.
4. La tasa de aprendizaje (lr) más baja (0.005) da mejores resultados en general.

In [ ]:
import matplotlib.pyplot as plt

# Crear etiquetas para el eje X: combinación de hiperparámetros como string
df_resultados['config'] = df_resultados.apply(
    lambda row: f"k={row['k']},lr={row['lr']},reg={row['reg']}", axis=1
)

# Ordenar por RMSE para una visualización clara
df_resultados = df_resultados.sort_values(by='RMSE')

# Crear gráfico de doble eje Y
fig, ax1 = plt.subplots(figsize=(12, 6))

ax2 = ax1.twinx()

# RMSE en eje Y izquierdo
ax1.plot(df_resultados['config'], df_resultados['RMSE'], 'o-', label='RMSE', color='tab:red')
ax1.set_ylabel('RMSE', color='tab:red')
ax1.tick_params(axis='y', labelcolor='tab:red')

# Precision@5 en eje Y derecho
ax2.plot(df_resultados['config'], df_resultados['Precision@5'], 's--', label='Precision@5', color='tab:blue')
ax2.set_ylabel('Precision@5', color='tab:blue')
ax2.tick_params(axis='y', labelcolor='tab:blue')

# Estética general
plt.title("Comparación de RMSE y Precision@5 según hiperparámetros")
ax1.set_xticks(range(len(df_resultados)))
ax1.set_xticklabels(df_resultados['config'], rotation=45, ha='right')
plt.tight_layout()
plt.grid(True)
plt.show()